태그별 인스타그램 유저 아이디 크롤링 함수 구현

In [11]:
import requests
import json
import urllib.request
import time
import pandas as pd

def insta_user_crawling(tag, folder_path):
  # 헤더 저장(자신의 로그인 정보를 넣으면 됩니다!)
  headers = {
              'cookie' : '',
              'user-agent': ''
              }
    
  user_list = []
  count = 0

  #태그별 인스타그램 유저 아이디 크롤링
  res = requests.get('https://www.instagram.com/explore/tags/{}/?__a=1'.format(tag), headers = headers)
  res_dics = json.loads(res.text)
  if len(res_dics) > 0:
    if res_dics['graphql']['hashtag']["edge_hashtag_to_media"]["page_info"]["has_next_page"] == True:
      first_end =  res_dics['graphql']['hashtag']["edge_hashtag_to_media"]["page_info"]["end_cursor"]
      end_cursor = first_end
      isTrue = True
      
      while isTrue:
        next_url = 'https://www.instagram.com/graphql/query/?query_hash=c769cb6c71b24c8a86590b22402fda50&variables=%7B"tag_name"%3A"{}"%2C"first"%3A2%2C"after"%3A"{}"%7D'.format(tag, end_cursor)
        res2 = requests.get(next_url)
        res2_dics = json.loads(res2.text)

        if len(res2_dics) > 0:
          if len(res2_dics['data']['hashtag']["edge_hashtag_to_media"]['edges']) > 0:
            for i in res2_dics['data']['hashtag']["edge_hashtag_to_media"]['edges']:
              count += 1
              if count%300 == 0:
                print(count)
                time.sleep(300) # 아이디 100개씩 크롤링 할 때마다 5분씩 쉬면서 동작
                print()
              user_list.append(i['node']['owner']['id'])
          else:
            print('해당 json 파일에 게시글 정보가 없습니다.')

          if res2_dics['data']['hashtag']["edge_hashtag_to_media"]["page_info"]["has_next_page"] == True:
            end_cursor =  res2_dics['data']['hashtag']["edge_hashtag_to_media"]["page_info"]["end_cursor"]  
            print(end_cursor) 
          else:
            isTrue = False
  else:
    print('해당 태그와 관련된 게시물이 없습니다.')

  print(user_list)

  #user_list csv파일로 저장
  unique_user_list = list(set(user_list)) #중복된 아이디 제거
  dataframe = pd.DataFrame(unique_user_list)
  dataframe.to_csv("{}/{}.csv".format(folder_path, tag), header=False, index = False)

insta_user_crawling 함수를 호출하여 크롤링 수행

In [ ]:
tag_list = ['대학생', '대학생공스타그램', '학사모', '개강'] #나이대와 관련된 태그 목록 작성

for i in tag_list:
  insta_user_crawling(i,"/content/drive/My Drive/insta_users/20대") #두번째 파라미터에 csv파일을 저장할 폴더를 입력